In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.7.0'

In [2]:
from tensorflow.keras.preprocessing.image import load_img

## Question - 1

In [3]:
model = keras.models.load_model('dogs_cats.h5')

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp9nkruskm/assets


In [5]:
!ls -lh|awk '{print $5 " " $6 " " $7 " " $8 " " $9}'

    
275 Dec 1 18:13 Dockerfile
8.5K Dec 1 18:16 Untitled.ipynb
4.0K Dec 1 18:11 __pycache__
86M Nov 25 12:32 dogs_cats.h5
43M Dec 1 18:17 dogs_cats.tflite
1.3K Dec 1 18:16 lambda_function.py
98K Oct 8 2014 pug.jpg


`Size of the converted model is 43 MB`

## Question - 2

In [6]:
import tensorflow.lite as tflite

In [7]:
interpreter = tflite.Interpreter(model_path='dogs_cats.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [8]:
output_index

13

## Question - 3

In [11]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [12]:
image = download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg')

In [13]:
prepared_image = prepare_image(image, (150,150))

In [14]:
def preprocess_input(x):
    x /= 255
    return x

In [15]:
x = np.array(prepared_image, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [16]:
X.shape

(1, 150, 150, 3)

In [17]:
X[0,0,0,0]

0.7058824

First Pixel is 0.7058824

## Question - 4

In [18]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [19]:
preds[0]

array([0.76990855], dtype=float32)

Prediction of the model is `0.76990855`

## Question - 5

Sending build context to Docker daemon  134.7MB

Step 1/5 : FROM agrigorev/zoomcamp-cats-dogs-lambda:v2
 ---> **322fc756f258**

Step 2/5 : RUN pip install pillow
 ---> Using cache
 ---> 0ea978b125d4

Step 3/5 : RUN pip install https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.7.0-cp38-cp38-linux_x86_64.whl
 ---> Using cache
 ---> e6f844100e36

Step 4/5 : COPY ["lambda_function.py", "./"]
 ---> 5163b1d76453

Step 5/5 : CMD ["lambda_function.lambda_handler"]
 ---> Running in 3c6289ba5643

Removing intermediate container 3c6289ba5643
 ---> ec7810b4e5e4

Successfully built ec7810b4e5e4
Successfully tagged mlzoomcamp_test2:latest


# Question - 6

In [21]:
import requests
url = 'http://localhost:8080/2015-03-31/functions/function/invocations'
data = {'url':'https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg'}

resp = requests.post(url, json=data).json()
resp

[0.5413472652435303]